# PyData SG Meet-up Jun 2016

1. Thomson Reuters Intelligent Tagging (Open Calais)
An open  semantic engine to tag entities - people, organizations, locations, events and etc. 
See: http://www.opencalais.com/


2. OpenPermID
An unifying identifier thread amongst different sources of the same entities.

See http://www.permid.org

For OpenPermID and OpenCalais, please obtain a token by registering at http://www.opencalais.com/opencalais-api/

3. Corelis DataFusion
Formerly Entagen pharmaceutical product, which was acquired by TR. See the video for an introduction.
https://www.youtube.com/watch?v=VkNG8TkEhBY



In [1]:
# Open Calais code 
# Brief description of code
# Feeds in input file in folder, then goes to the OpenCalais API to tag the document
# Output is then saved into the output folder; 

import os
import requests
import os

calais_url = 'https://api.thomsonreuters.com/permid/calais'

def main():

    # input files in txt format
    input_file = "C:\\input\\"
    output_dir = "C:\\output\\"
    # Please obtain a token through the website.
    access_token = "yourtoken"   

    if not os.path.exists(input_file):
        print 'The file [%s] does not exist' % input_file
        return
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    headers = {'X-AG-Access-Token' : access_token, 'Content-Type' : 'text/raw', 'outputformat' : 'application/json'}
    sendFiles(input_file, headers, output_dir)

def sendFiles(files, headers, output_dir):
    is_file = os.path.isfile(files)
    if is_file == True:
        sendFile(files, headers, output_dir)
    else:
        for file_name in os.listdir(files):
            fname = os.getcwd()+"\\input\\"+file_name
            if os.path.isfile(fname):
                sendFile(fname, headers, output_dir)


def sendFile(file_name, headers, output_dir):
    files = {'file': open(file_name, 'rb')}
    response = requests.post(calais_url, files=files, headers=headers, timeout=80)
    print 'status code: %s' % response.status_code
    content = response.text
    print 'Results received: %s' % content
    if response.status_code == 200:
        saveFile(file_name, output_dir, content)

def saveFile(file_name, output_dir, content):
    output_file_name = os.path.basename(file_name) + '.xml'
    output_file = open(os.path.join(output_dir, output_file_name), 'wb')
    output_file.write(content.encode('utf-8'))
    output_file.close()

if __name__ == "__main__":
   main()

status code: 200
Results received: {"doc":{"info":{"calaisRequestID":"39dd9674-312f-6cc2-1554-3c2a90c249f6","id":"http:\/\/id.opencalais.com\/5oTm9tj2oI0LDfFRW3*KEA","ontology":"http:\/\/mdaast-virtual-onecalais.int.thomsonreuters.com\/owlschema\/9.5\/onecalais.owl.allmetadata.xml","docId":"http:\/\/d.opencalais.com\/dochash-1\/1b83b703-a6b4-3e53-a026-90fcb3b7ca94","document":"--c8bb295045cf4b96b923b49c881f15e9 \nContent-Disposition: form-data; name=\"file\"; filename=\"trit_1.txt\" \n \nBeijing is �satisfied� with the work of Hong Kong�s leader Leung Chun-ying, according to Zhang Dejiang, China�s third highest level official who commented on the chief executive months before an election for the top job is to take place. \nZhang told dozens of Hong Kong officials �the central government is satisfied with the work of the chief executive and of the SAR government� and noted that President Xi Jinping and Premier Li Keqiang expressed �full endorsement� for Leung during his annual duty visi

In [ ]:
# Code takes in input entity by name;
# Goes to the PermID.org website to search for closely matched entities and returns in permids in a list
#

import requests, re

def htmlparse(url):
    urlr=re.sub(':', '%3A',url)
    urlr=re.sub('/', '%2F',urlr)
    urlr=re.sub(' ', '%20',urlr)
    return urlr

def searchPermID(entid, iid=1):
    url = "https://api.thomsonreuters.com:443/permid/search?"
    typed="&Entitytype=Organization"    
    if iid==1:
        typed="&Entitytype=Organization"
    elif iid==2:
        typed="&Entitytype=Person"
    else:
        typed="&Entitytype=Quote"
        
    url = url+ "q=name%3A" + htmlparse(str(entid))
    # search by name, ticker, RIC
    url = url + typed
    token='yourtoken'
    print "PermID url is " + str(url)

    headers = { 'X-AG-Access-Token' :  token } 
    response = requests.get(url, headers=headers)
    
    try:
        if response.status_code <> 200:
            raise ValueError('Error in HTTP ')
        else:
            res = response.json()  # output is in Python list
            print 'HTTP request is successful with code ' + str(response.status_code )
            return res['result']
    except:
        print 'Error in HTTP call ' + str(response.status_code )
        
if __name__ == "__main__":   
   res=searchPermID("YourSearchEntity")